In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
# import dataload
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 0. 데이터 로드

In [4]:
cd /content/drive/MyDrive/2021/날씨 공모전/code

/content/drive/MyDrive/2021/날씨 공모전/code


## 0-1. 날씨

#### 0-1-1. 지역별

In [ ]:
weather = pd.read_csv('../data/weather_final2.csv', encoding='cp949')
weather['날짜'] = weather['날짜'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))

weather.head()

,날짜,지점번호,평균기온,최고기온,최저기온,일별강수량,1시간최대강수량,평균풍속,지역,PM10,PM25,1시간최대습도,일조시간합,최고현지기압,연,월,일,분기,요일,공휴일명,공휴일여부,주말여부,계절,체감온도,열지수,폭염여부,강수여부,year,month
0,2018-01-01,105,1.3,5.7,-2.1,0.0,0.0,3.7,강릉,20.066667,13.400000,25.4,57.9,1023.0,2018,1,1,1,0,1월1일,1,0,3,7.502056,-39.701524,0,0,2018,1
1,2018-01-01,112,-0.3,2.7,-2.7,0.0,0.0,1.6,인천,37.518681,18.641758,67.2,53.8,1020.3,2018,1,1,1,0,1월1일,1,0,3,9.992379,-42.924922,0,0,2018,1
2,2018-01-01,119,-1.7,4.7,-6.9,0.0,0.0,1.0,수원,42.782895,21.375000,84.7,52.7,1025.1,2018,1,1,1,0,1월1일,1,0,3,10.136402,-45.791893,0,0,2018,1
3,2018-01-01,136,-1.0,4.7,-6.5,0.0,0.0,2.2,안동,39.750000,28.583333,56.4,58.3,1010.6,2018,1,1,1,0,1월1일,1,0,3,8.356692,-44.375704,0,0,2018,1
4,2018-01-01,152,2.1,6.2,-0.4,0.0,0.0,3.3,울산,38.548193,15.433735,42.8,58.3,1017.1,2018,1,1,1,0,1월1일,1,0,3,8.861429,-38.064334,0,0,2018,1


#### 0-1-2. 날씨지수

In [ ]:
'''
weather_idx = pd.read_csv('../data/data_with_weather.csv', encoding='cp949')
weather_idx['date'] = weather_idx['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
weather_idx.head()

,date,sex,age,big_cat,sm_cat,qty,cnt,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa
0,2018-01-01,F,20,식품,가공란,37,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
1,2018-01-01,F,30,식품,가공란,16,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
2,2018-01-01,F,40,식품,가공란,9,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
3,2018-01-01,F,50,식품,가공란,3,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
4,2018-01-01,M,20,식품,가공란,13,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385


## 0-2. 구매이력 및 검색량

In [ ]:
'''
trend_weather = pd.read_csv('../data/trend_with_weather.csv', encoding='cp949')
trend_weather['date'] = trend_weather['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
trend_weather.head()

,date,sex,age,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보
0,2018-01-01,F,20,식품,가공란,37.0,0.0,10.69387,4.09137,0.00000,0.00000,0.04440
1,2018-01-01,F,30,식품,가공란,16.0,0.0,12.42969,6.35294,0.00000,0.51282,0.42417
2,2018-01-01,F,40,식품,가공란,9.0,0.0,13.02249,4.88037,0.00000,0.40650,0.00000
3,2018-01-01,F,50,식품,가공란,3.0,0.0,11.69080,4.53350,1.21951,0.90909,0.00000
4,2018-01-01,M,20,식품,가공란,13.0,0.0,11.23032,3.61296,0.38910,2.22222,0.04060


In [ ]:
# trend_weather.shape

(2568668, 12)

## 0-3. 공휴일

In [ ]:
'''
calender = weather[['날짜', '분기', '요일', '공휴일명', '공휴일여부', '주말여부', '계절']]
calender.head()

,날짜,분기,요일,공휴일명,공휴일여부,주말여부,계절
0,2018-01-01,1,0,1월1일,1,0,3
1,2018-01-01,1,0,1월1일,1,0,3
2,2018-01-01,1,0,1월1일,1,0,3
3,2018-01-01,1,0,1월1일,1,0,3
4,2018-01-01,1,0,1월1일,1,0,3


## 0-4. 클러스터 정보

In [ ]:
food_cluster = pd.read_csv('../data/food_tsclustering_norm_3.csv', encoding='cp949')[['sm_cat', 'cluster']]
food_cluster.head()

,sm_cat,cluster
0,가공란,2
1,가자미,1
2,갈비/찜/바비큐용 돈육,1
3,갈비용 우육,0
4,갈치,0


# 1. 데이터 preprocessing

In [ ]:
'''
data = trend_weather.merge(weather_idx.drop(['qty'], axis=1), how='left', on=['date', 'sex', 'age', 'big_cat', 'sm_cat'])
data = data.merge(calender.groupby(['날짜']).mean().reset_index(), left_on='date', right_on='날짜')
data.shape
'''

(2568668, 30)

In [ ]:
data = pd.read_csv('../data/final_0606.csv', encoding='cp949').iloc[:,1:]
data['date'] = data['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))
print(data.shape)
data.head()

(2568668, 30)


,date,sex,age,big_cat,sm_cat,qty,cnt,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,분기,요일,공휴일명,공휴일여부,주말여부,계절
0,2018-01-01,F,20,식품,가공란,37.0,0.480964,0.0,10.69387,4.09137,0.00000,0.00000,0.04440,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3
1,2018-01-01,F,30,식품,가공란,16.0,0.480964,0.0,12.42969,6.35294,0.00000,0.51282,0.42417,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3
2,2018-01-01,F,40,식품,가공란,9.0,0.480964,0.0,13.02249,4.88037,0.00000,0.40650,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3
3,2018-01-01,F,50,식품,가공란,3.0,0.480964,0.0,11.69080,4.53350,1.21951,0.90909,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3
4,2018-01-01,M,20,식품,가공란,13.0,0.480964,0.0,11.23032,3.61296,0.38910,2.22222,0.04060,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1월1일,1,0,3


In [ ]:
def region_weather(지역):
    weather_true = weather.drop(['연','월', '일', '분기', '요일', '공휴일명', '공휴일여부', '주말여부', '계절'], axis=1) # weather_final
    weather1 = pd.pivot_table(weather_true[weather_true['지역'].isin(지역)], index='날짜', columns='지역').reset_index(drop=True)
    weather1.columns = [l+"_"+w for l,w in zip(weather1.columns.get_level_values(1), weather1.columns.get_level_values(0))]
    weather1['날짜'] = weather['날짜'].unique()
    weather1['연'] = weather['날짜'].apply(lambda x : x.year)
    weather1['월'] = weather['날짜'].apply(lambda x : x.month)
    return weather1

In [ ]:
def get_dummies(data, var_list):
    # var_list = ['var1', 'var2']
    for var in var_list:
        if var=='sex':
            data[var] = pd.get_dummies(data[var]).iloc[:,:1] # F(1), M(0)
        else:
            data = pd.concat([data.drop(var, axis=1), pd.get_dummies(data[var], prefix=var, drop_first=True)], axis=1)
    return data

In [ ]:
food = data[data.big_cat=='식품']
food = food.merge(food_cluster, on='sm_cat')
print("food_full's shape : {}".format(food.shape))

food0 = food[food.cluster==0].reset_index(drop=True).drop(['big_cat', '공휴일명', 'cluster'], axis=1)
food1 = food[food.cluster==1].reset_index(drop=True).drop(['big_cat', '공휴일명', 'cluster'], axis=1)
food2 = food[food.cluster==2].reset_index(drop=True).drop(['big_cat', '공휴일명', 'cluster'], axis=1)

# 더미변수화 
food0 = get_dummies(food0, ['age', 'sex', 'sm_cat'])
food1 = get_dummies(food1, ['age', 'sex', 'sm_cat'])
food2 = get_dummies(food2, ['age', 'sex', 'sm_cat'])

food0 = pd.concat([food0[['date', 'qty']], food0.drop(['date', 'qty'], axis=1)], axis=1)

# food0 = food0.merge(region_weather(['서울', '부산', '제주']), how='left', left_on='date', right_on='날짜').drop(['날짜'], axis=1)
# food1 = food1.merge(region_weather(['서울', '부산', '수원']), how='left', left_on='date', right_on='날짜').drop(['날짜'], axis=1)
# food2 = food2.merge(region_weather(['서울', '부산', '대구']), how='left', left_on='date', right_on='날짜').drop(['날짜'], axis=1)

print("food cluster0's shape : {}".format(food0.shape))
print("food cluster1's shape : {}".format(food1.shape))
print("food cluster2's shape : {}".format(food2.shape))

food_full's shape : (1496505, 31)
food cluster0's shape : (403084, 86)
food cluster1's shape : (574772, 110)
food cluster2's shape : (518649, 103)


In [ ]:
food0.columns

Index(['date', 'qty', 'sex', 'cnt', 'ratio', '오늘날씨', '미세먼지', '오늘 비', '기상정보',
       '태풍정보', 'avg_ta', 'max_ta', 'min_ta', 'rn_day', 'rn_hr1', 'avg_ws',
       'PM10', 'PM25', 'hm_max', 'sum_ss_hr', 'max_pa', '분기', '요일', '공휴일여부',
       '주말여부', '계절', 'age_30', 'age_40', 'age_50', 'age_60', 'sm_cat_갈치',
       'sm_cat_감/홍시', 'sm_cat_감귤/한라봉/오렌지', 'sm_cat_감말랭이', 'sm_cat_감자',
       'sm_cat_건대추', 'sm_cat_건어물 멸치', 'sm_cat_건어물 황태', 'sm_cat_게장류',
       'sm_cat_견과류 밤', 'sm_cat_곡물차', 'sm_cat_곶감/반건시', 'sm_cat_과일세트',
       'sm_cat_국내산 돈육', 'sm_cat_굴 생물', 'sm_cat_굴비/조기', 'sm_cat_대게/킹크랩',
       'sm_cat_돼지 곱창', 'sm_cat_딸기/복분자/블루베리', 'sm_cat_랍스타', 'sm_cat_명태/동태',
       'sm_cat_무/배추', 'sm_cat_미나리', 'sm_cat_밀크티/티라떼', 'sm_cat_생선류',
       'sm_cat_소고기 등심/안심', 'sm_cat_소고기 육회', 'sm_cat_수산 생물', 'sm_cat_시금치',
       'sm_cat_양념우육', 'sm_cat_옥돔', 'sm_cat_옥수수', 'sm_cat_유자차', 'sm_cat_율무차',
       'sm_cat_이온음료', 'sm_cat_전복 생물', 'sm_cat_전통주', 'sm_cat_절임배추/김치속',
       'sm_cat_조개', 'sm_cat_차 선물세트', 'sm_cat_참외/메론/

In [ ]:
food0.head()

,date,qty,sex,cnt,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,분기,요일,공휴일여부,주말여부,계절,age_30,age_40,age_50,age_60,sm_cat_갈치,sm_cat_감/홍시,sm_cat_감귤/한라봉/오렌지,sm_cat_감말랭이,sm_cat_감자,sm_cat_건대추,sm_cat_건어물 멸치,sm_cat_건어물 황태,sm_cat_게장류,sm_cat_견과류 밤,...,sm_cat_대게/킹크랩,sm_cat_돼지 곱창,sm_cat_딸기/복분자/블루베리,sm_cat_랍스타,sm_cat_명태/동태,sm_cat_무/배추,sm_cat_미나리,sm_cat_밀크티/티라떼,sm_cat_생선류,sm_cat_소고기 등심/안심,sm_cat_소고기 육회,sm_cat_수산 생물,sm_cat_시금치,sm_cat_양념우육,sm_cat_옥돔,sm_cat_옥수수,sm_cat_유자차,sm_cat_율무차,sm_cat_이온음료,sm_cat_전복 생물,sm_cat_전통주,sm_cat_절임배추/김치속,sm_cat_조개,sm_cat_차 선물세트,sm_cat_참외/메론/수박,sm_cat_코코아/핫초코,sm_cat_탄산음료,sm_cat_토마토,sm_cat_표고버섯,sm_cat_프로폴리스/로얄젤리,sm_cat_한우육,sm_cat_해초류,sm_cat_혼합견과,sm_cat_혼합곡,sm_cat_홍삼 분말/환,sm_cat_홍삼 음료,sm_cat_홍삼/인삼 제품,sm_cat_홍삼액/홍삼정,sm_cat_홍삼절편/홍삼정과,sm_cat_회
0,2018-01-01,1.0,1,0.0,14.81481,10.69387,4.09137,0.00000,0.00000,0.04440,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2018-01-01,0.0,1,0.0,12.53561,12.42969,6.35294,0.00000,0.51282,0.42417,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2018-01-01,1.0,1,0.0,11.27819,13.02249,4.88037,0.00000,0.40650,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2018-01-01,0.0,1,0.0,9.61538,11.69080,4.53350,1.21951,0.90909,0.00000,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2018-01-01,0.0,0,0.0,22.37442,11.23032,3.61296,0.38910,2.22222,0.04060,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 2. 모델링

In [ ]:
! pip install shap

In [ ]:
import shap
import random
import xgboost as xgb
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor

## 2-1. baseline

### 2-1-1. food0

In [ ]:
def sampling(data):
    random.seed(1234)
    # 랜덤으로 요일 선택
    sample_day = []; sample_idx = []
    count = 730//7 
    day_list = day_list = [1,2,3,4,5,6,7]
    sample_list = [random.choice(day_list) for i in range(count)]

    # 랜덤으로 선택된 요일의 날짜
    start = datetime.datetime(2018, 1, 1)
    for days in sample_list:
        sample_day.append(start + datetime.timedelta(days = days))
        start = start + datetime.timedelta(weeks = 1)

    # 샘플링
    for date in sample_day:
        sample_idx.extend(data[data['date']==date].index)
        
    all_idx = data.index.tolist()
    train_idx = list(set(all_idx) - set(sample_idx))
    train = data.iloc[train_idx].reset_index(drop=True)
    test = data.iloc[sample_idx].reset_index(drop=True)
    return train.drop('date', axis=1), test.drop('date', axis=1)

def MAPE(y_test, y_pred):
	return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [ ]:
food0.head()

In [ ]:
# train and test split 
train, test = sampling(food0)

# standardization 
std_scaler = StandardScaler()
train_std = std_scaler.fit_transform(train); test_std = std_scaler.transform(test)

# X, y split
X_train = train_std[:,2:]; y_train = train_std[:,1:2]
X_test = test_std[:,2:]; y_test = test_std[:,1:2]

# batch index
kf = KFold(n_splits=3, random_state=1234, test_size=0)

TypeError: ignored

In [ ]:
train_rmse = []
test_mape = []

batch_num = 1
for batch_idx, _ in kf.split(X_train):
    # data
    X_batch = X_train[batch_idx]; y_batch = y_train[batch_idx,:]

    # model
    xgb_model = xgb.XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100) 
    xgb_model.fit(X_batch, y_batch)
    
    # test
    y_batch_pred = xgb_model.predict(X_batch)
    rmse = mean_squared_error(y_batch, y_batch_pred)
    mape = MAPE(y_batch, y_batch_pred)
    train_rmse.append(rmse)
    train_rmse.append(mape)
    print('========= {}th training results =========')
    print('RMSE : '.format(rmse))
    print('MAPE : '.format(mape))
    batch_num += 1

NameError: ignored

In [ ]:
y_pred = xgb_model.predict(X_test)
print(mean_squared_error(y_test, y_pred))

plt.figure(figsize=(12,4))
plt.plot(y_test, label='true')
plt.plot(y_pred, label='pred')
plt.legend(loc = 'best')
plt.show()